In [1]:
#coding=utf-8
import pandas as pd
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Input
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras.utils import to_categorical
%matplotlib inline

Using TensorFlow backend.


In [2]:
def normalize(df):
    norm = df.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return norm
def train_windows(df, ref_day=5, predict_day=1):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day):
        X_train.append(np.array(df.iloc[i:i+ref_day,:-1]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day]["y"]))
    return np.array(X_train), np.array(Y_train)
def lstm_stock_model(shape):
    model = Sequential()
    model.add(LSTM(256, input_shape=(shape[1], shape[2]), return_sequences=True))
    model.add(LSTM(256, return_sequences=True))
    model.add(TimeDistributed(Dense(1)))
    model.add(Flatten())
    model.add(Dense(5,activation='linear'))
    model.add(Dense(1,activation='linear'))
    model.compile(loss="mean_absolute_error", optimizer="adam",metrics=['mean_absolute_error'])
    model.summary()
    return model

In [3]:
def sum(f):
    #train = pd.read_csv('Apple.csv')
    train = f.dropna()
    train['y'] = train['Close'].shift(-1)
    #train = type_result(train)
    train.iloc[:,2:4] = normalize(train.iloc[:,2:4])
    train['Date'] = pd.to_datetime(train['Date'], format='%Y%m%d')
    train['weekday'] = train['Date'].apply(lambda x: x.day)
    train['year'] = train['Date'].apply(lambda x: x.year)
    train['Month'] = train['Date'].apply(lambda x: x.month)
    test = train[-30:]
    train = train[(train['year']==2019)|((train['year']==2018)&(train['Month']<=3))|((train['year']==2018)&(train['Month']==4)&(train['weekday']<=27))]
    #train & test
    train = train.reset_index()
    test = test.reset_index()
    train = train.dropna()
    train.drop(['High','Min','Open','Date','weekday','year','Month',"index"], axis=1, inplace=True)
    test.drop(['Date','High','Min','Open','weekday','year','Month','index'], axis=1, inplace=True)
    X_train, Y_train = train_windows(train, 5, 1)
    X_test, Y_test = train_windows(test, 5, 1)
    model = lstm_stock_model(X_train.shape)
    callback = EarlyStopping(monitor="mean_absolute_error", patience=10, verbose=1, mode="auto")
    history = model.fit(X_train, Y_train, epochs=10, batch_size=5, validation_split=0.1, callbacks=[callback],shuffle=True)

    dense1_layer_model = Model(inputs=model.input,outputs=model.layers[5].output)
    yguess = dense1_layer_model.predict(X_test)
    del dense1_layer_model
    del model
    
    return yguess

In [4]:
result=[]
#file = pd.read_csv('Apple.csv')
res=0
for i in range(20):
    if i==0:
        file = pd.read_csv('Intel.csv')
    if i==1:
        file = pd.read_csv('AMD.csv')
    if i==2:
        file = pd.read_csv('Cisco.csv')
    if i==3:
        file = pd.read_csv('Apple.csv')
    if i==4:
        file = pd.read_csv('Micron.csv')
    if i==5:
        file = pd.read_csv('Nvidia.csv')
    if i==6:
        file = pd.read_csv('Qualcomm.csv')
    if i==7:
        file = pd.read_csv('Amazon.csv')
    if i==8:
        file = pd.read_csv('Netflix.csv')
    if i==9:
        file = pd.read_csv('Facebook.csv')
    if i==10:
        file = pd.read_csv('Google.csv')
    if i==11:
        file = pd.read_csv('Alibaba.csv')
    if i==12:
        file = pd.read_csv('eBay.csv')
    if i==13:
        file = pd.read_csv('IBM.csv')
    if i==14:
        file = pd.read_csv('Xilinx.csv')
    if i==15:
        file = pd.read_csv('Texas.csv')
    if i==16:
        file = pd.read_csv('Nokia.csv')
    if i==17:
        file = pd.read_csv('Tesla.csv')
    if i==18:
        file = pd.read_csv('Microsoft.csv')
    if i==19:
        file = pd.read_csv('Synopsys.csv')
    
    res=sum(file)
    ref=file['Close'][42]
    if res[0]>ref:
        result.append(0)
    elif res[0]==ref:
        result.append(1)
    else:
        result.append(2) 
    print(ref)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 5, 256)            265216    
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 256)            525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 1)              257       
_________________________________________________________________
flatten_1 (Flatten)          (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 790,821
Trainable params: 790,821
Non-trainable params: 

Epoch 6/10
32/32 [==============================] - 0s 11ms/step - loss: 25.9036 - mean_absolute_error: 25.9036 - val_loss: 22.9935 - val_mean_absolute_error: 22.9935
Epoch 7/10
32/32 [==============================] - 0s 10ms/step - loss: 19.6457 - mean_absolute_error: 19.6457 - val_loss: 16.3338 - val_mean_absolute_error: 16.3338
Epoch 8/10
32/32 [==============================] - 0s 8ms/step - loss: 12.7214 - mean_absolute_error: 12.7214 - val_loss: 8.9833 - val_mean_absolute_error: 8.9833
Epoch 9/10
32/32 [==============================] - 0s 8ms/step - loss: 5.0809 - mean_absolute_error: 5.0809 - val_loss: 0.9844 - val_mean_absolute_error: 0.9844
Epoch 10/10
32/32 [==============================] - 0s 8ms/step - loss: 1.2704 - mean_absolute_error: 1.2704 - val_loss: 0.9900 - val_mean_absolute_error: 0.9900
54.94
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 5, 

Epoch 4/10
32/32 [==============================] - 0s 8ms/step - loss: 168.1789 - mean_absolute_error: 168.1789 - val_loss: 163.5870 - val_mean_absolute_error: 163.5870
Epoch 5/10
32/32 [==============================] - 0s 8ms/step - loss: 163.1569 - mean_absolute_error: 163.1569 - val_loss: 158.7427 - val_mean_absolute_error: 158.7427
Epoch 6/10
32/32 [==============================] - 0s 8ms/step - loss: 158.2034 - mean_absolute_error: 158.2034 - val_loss: 153.5040 - val_mean_absolute_error: 153.5040
Epoch 7/10
32/32 [==============================] - 0s 8ms/step - loss: 152.6782 - mean_absolute_error: 152.6782 - val_loss: 147.4718 - val_mean_absolute_error: 147.4718
Epoch 8/10
32/32 [==============================] - 0s 8ms/step - loss: 146.2922 - mean_absolute_error: 146.2922 - val_loss: 140.5574 - val_mean_absolute_error: 140.5574
Epoch 9/10
32/32 [==============================] - 0s 8ms/step - loss: 139.0815 - mean_absolute_error: 139.0815 - val_loss: 132.8994 - val_mean_absol

32/32 [==============================] - 5s 141ms/step - loss: 363.3782 - mean_absolute_error: 363.3782 - val_loss: 372.2838 - val_mean_absolute_error: 372.2838
Epoch 2/10
32/32 [==============================] - 0s 8ms/step - loss: 359.3813 - mean_absolute_error: 359.3813 - val_loss: 365.9924 - val_mean_absolute_error: 365.9924
Epoch 3/10
32/32 [==============================] - 0s 7ms/step - loss: 350.2755 - mean_absolute_error: 350.2755 - val_loss: 350.0406 - val_mean_absolute_error: 350.0406
Epoch 4/10
32/32 [==============================] - 0s 7ms/step - loss: 330.3635 - mean_absolute_error: 330.3635 - val_loss: 326.8722 - val_mean_absolute_error: 326.8722
Epoch 5/10
32/32 [==============================] - 0s 8ms/step - loss: 305.6500 - mean_absolute_error: 305.6500 - val_loss: 299.5570 - val_mean_absolute_error: 299.5570
Epoch 6/10
32/32 [==============================] - 0s 8ms/step - loss: 277.9722 - mean_absolute_error: 277.9722 - val_loss: 272.6245 - val_mean_absolute_error

Train on 32 samples, validate on 4 samples
Epoch 1/10
32/32 [==============================] - 5s 168ms/step - loss: 182.7524 - mean_absolute_error: 182.7524 - val_loss: 187.0491 - val_mean_absolute_error: 187.0491
Epoch 2/10
32/32 [==============================] - 0s 8ms/step - loss: 180.8869 - mean_absolute_error: 180.8869 - val_loss: 183.3767 - val_mean_absolute_error: 183.3767
Epoch 3/10
32/32 [==============================] - 0s 8ms/step - loss: 176.4107 - mean_absolute_error: 176.4107 - val_loss: 178.3037 - val_mean_absolute_error: 178.3037
Epoch 4/10
32/32 [==============================] - 0s 8ms/step - loss: 170.7354 - mean_absolute_error: 170.7354 - val_loss: 170.9217 - val_mean_absolute_error: 170.9217
Epoch 5/10
32/32 [==============================] - 0s 8ms/step - loss: 161.7937 - mean_absolute_error: 161.7937 - val_loss: 159.7433 - val_mean_absolute_error: 159.7433
Epoch 6/10
32/32 [==============================] - 0s 8ms/step - loss: 149.5334 - mean_absolute_error: 1

Train on 32 samples, validate on 4 samples
Epoch 1/10
32/32 [==============================] - 7s 212ms/step - loss: 121.8289 - mean_absolute_error: 121.8289 - val_loss: 98.9343 - val_mean_absolute_error: 98.9343
Epoch 2/10
32/32 [==============================] - 0s 8ms/step - loss: 99.7407 - mean_absolute_error: 99.7407 - val_loss: 72.3613 - val_mean_absolute_error: 72.3613
Epoch 3/10
32/32 [==============================] - 0s 8ms/step - loss: 73.6264 - mean_absolute_error: 73.6264 - val_loss: 50.7272 - val_mean_absolute_error: 50.7272
Epoch 4/10
32/32 [==============================] - 0s 8ms/step - loss: 55.3971 - mean_absolute_error: 55.3971 - val_loss: 36.2807 - val_mean_absolute_error: 36.2807
Epoch 5/10
32/32 [==============================] - 0s 8ms/step - loss: 41.7919 - mean_absolute_error: 41.7919 - val_loss: 23.1182 - val_mean_absolute_error: 23.1182
Epoch 6/10
32/32 [==============================] - 0s 8ms/step - loss: 28.3956 - mean_absolute_error: 28.3956 - val_loss: 

Train on 32 samples, validate on 4 samples
Epoch 1/10
32/32 [==============================] - 8s 238ms/step - loss: 269.9633 - mean_absolute_error: 269.9633 - val_loss: 235.7513 - val_mean_absolute_error: 235.7513
Epoch 2/10
32/32 [==============================] - 0s 8ms/step - loss: 265.4346 - mean_absolute_error: 265.4346 - val_loss: 229.5752 - val_mean_absolute_error: 229.5752
Epoch 3/10
32/32 [==============================] - 0s 9ms/step - loss: 258.1001 - mean_absolute_error: 258.1001 - val_loss: 220.7585 - val_mean_absolute_error: 220.7585
Epoch 4/10
32/32 [==============================] - 0s 9ms/step - loss: 249.2506 - mean_absolute_error: 249.2506 - val_loss: 212.3509 - val_mean_absolute_error: 212.3509
Epoch 5/10
32/32 [==============================] - 0s 10ms/step - loss: 240.9381 - mean_absolute_error: 240.9381 - val_loss: 203.8785 - val_mean_absolute_error: 203.8785
Epoch 6/10
32/32 [==============================] - 0s 9ms/step - loss: 232.0976 - mean_absolute_error: 

In [5]:
print(result)

[0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 2]


In [6]:
file = open("0506_0780207.txt","w") 
for index in range(20):
    file.write(str(index+1))
    file.write("  ")
    file.write(str(result[index]))
    file.write("\n")
 
file.close()